<a href="https://colab.research.google.com/github/Taishi-N324/CPP/blob/main/t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

!pip install torch==1.11.0 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html



!pip install --upgrade pip
!pip install transformers["ja"] numpy pandas sentencepiece fairseq
!pip install -U jupyter ipywidgets 


Looking in links: https://download.pytorch.org/whl/torch_stable.html
tcmalloc: large alloc 1412186112 bytes == 0x3666000 @  0x7ff90b4be1e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x59aeca 0x515655 0x593dd7 0x511e2c 0x593dd7 0x511e2c 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x4bca8a 0x59c019
tcmalloc: large alloc 1412186112 bytes == 0x5792a000 @  0x7ff90b4be1e7 0x4a3940 0x5b438c 0x64cfe7 0x59b076 0x515655 0x549576 0x593fce 0x511e2c 0x593dd7 0x511e2c 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6
tcmalloc: large alloc 1412186112 bytes == 0xabbee000 @  0x7ff90b4be1e7 0x4a3940 0x59b5e2 0x63a515 0x63bd66 0x63be16 0x59afff 0x515655 0x593dd7 0x511e2c 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x549576 0x593fce 0x511e2c 0x593dd7

In [2]:
!wget "https://nlp.stanford.edu/projects/jesc/data/split.tar.gz"
!tar -zxvf split.tar.gz
!ls split

--2022-05-15 05:50:44--  https://nlp.stanford.edu/projects/jesc/data/split.tar.gz
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102198329 (97M) [application/x-gzip]
Saving to: ‘split.tar.gz.2’

split.tar.gz.2      100%[===================>]  97.46M  27.4MB/s    in 5.5s    

2022-05-15 05:50:49 (17.9 MB/s) - ‘split.tar.gz.2’ saved [102198329/102198329]

split/
split/train
split/dev
split/test
dev  test  train


In [3]:
res = []
for line in open('split/train', 'r', encoding='utf-8'):
    text = line.split('\t')
    text = [t.rstrip('\n') for t in text]
    res.extend(text)
for line in open('split/dev', 'r', encoding='utf-8'):
    text = line.split('\t')
    text = [t.rstrip('\n') for t in text]
    res.extend(text)
for line in open('split/test', 'r', encoding='utf-8'):
    text = line.split('\t')
    text = [t.rstrip('\n') for t in text]
    res.extend(text)

print(len(res))
with open('tmp.txt', 'w') as f:
    for d in res:
        f.write("%s\n" % d)

5602776


In [4]:
!tail tmp.txt

am i free to go?
行っていいのですか?
you'll definitely become a good nurse.
あんた きっと いい看護師になるよ。
and isn't that what your facemash was about?
フェイスマッシュも同じだ
what is it this time?
ロケットパンチ止めようとしてんぞ! どういう状況だ? これ!
you poured your father's remains in and closed it.
一度 封をはがして灰を入れ また封をしても―


In [5]:
!pip install sentencepiece

In [6]:
import sentencepiece 

In [ ]:
sentencepiece.SentencePieceTrainer.Train("--input=tmp.txt --model_prefix=new_spm_model --vocab_size=10000 --vocabulary_output_piece_score=false --model_type=bpe")

In [ ]:
!wget "https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz"
!tar -zxvf mbart.cc25.v2.tar.gz
!ls mbart.cc25.v2

In [ ]:
edited = []
for line in open("new_spm_model.vocab", 'r', encoding='utf-8'):
    if line in ["<unk>\n", "<s>\n", "</s>\n"]:
        continue
    new_line = line.rstrip('\n') + " 1\n"
    edited.append(new_line)

with open('new_dict.txt', 'w') as f:
    for e in edited:
        f.write(e)

In [ ]:
!mkdir reduced_model
!ls

In [ ]:
from fairseq.data import Dictionary
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer, MBartConfig
)
from typing import List
import torch

In [ ]:
langs = [
    "ar_AR",
    "cs_CZ",
    "de_DE",
    "en_XX",
    "es_XX",
    "et_EE",
    "fi_FI",
    "fr_XX",
    "gu_IN",
    "hi_IN",
    "it_IT",
    "ja_XX",
    "kk_KZ",
    "ko_KR",
    "lt_LT",
    "lv_LV",
    "my_MM",
    "ne_NP",
    "nl_XX",
    "ro_RO",
    "ru_RU",
    "si_LK",
    "tr_TR",
    "vi_VN",
    "zh_CN"
]

def load_dict(langs: List[str], path: str) -> Dictionary:
    d = Dictionary.load(path)
    for ll in langs:
        d.add_symbol(f"[{ll}]")
    d.add_symbol("<mask>")
    d.add_symbol("<pad>")
    return d


pre_dict = load_dict(langs, "./mbart.cc25.v2/dict.txt")
ft_dict = load_dict(langs, "./new_dict.txt")

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")
org_sd = model.state_dict()
resized_sd = model.state_dict()

mapping: List[int] = []
for i in range(len(ft_dict)):
    word = ft_dict[i]
    mapping.append(pre_dict.index(word))

for name in ["model.encoder.embed_tokens.weight", "model.decoder.embed_tokens.weight", "model.shared.weight", "lm_head.weight"]:
    pre_tensor: torch.Tensor = org_sd[name]
    ft_tensor = torch.zeros(
        [len(ft_dict), 1024], dtype=pre_tensor.dtype, layout=pre_tensor.layout, device=pre_tensor.device,
    )
    for ft_i, pre_i in enumerate(mapping):
        ft_tensor[ft_i] = pre_tensor[pre_i]
    resized_sd[name] = ft_tensor
resized_sd["final_logits_bias"] = resized_sd["final_logits_bias"][:, :len(ft_dict)]

config = MBartConfig.from_pretrained("facebook/mbart-large-cc25")
config.vocab_size = len(ft_dict)
print(config)


In [ ]:
new_model = MBartForConditionalGeneration.from_pretrained(None, config=config, state_dict=resized_sd)
#new_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-en-ro")

#model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-en-ro")

In [ ]:
new_model.save_pretrained("./reduced_model")

In [ ]:
!ls reduced_model

In [ ]:
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")

tokenizer.save_pretrained("./reduced_model")

In [ ]:
!mv ./new_spm_model.model ./reduced_model/sentencepiece.bpe.model
!ls -al ./reduced_model

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("./reduced_model")
tokenizer = MBartTokenizer.from_pretrained("./reduced_model")

In [ ]:
!mkdir output

In [ ]:
from transformers import (
    Seq2SeqTrainingArguments, Seq2SeqTrainer
)
import numpy as np
import re

result_dir = "./output"

In [ ]:
def data_collator(features: list):
    x = [f["translation"]["ja"] for f in features]
    y = [f["translation"]["en"] for f in features]
    inputs = tokenizer(x, return_tensors="pt", padding='max_length', truncation=True, max_length=32)
    with tokenizer.as_target_tokenizer():
        inputs['labels'] = tokenizer(y, return_tensors="pt", padding='max_length', truncation=True, max_length=48)['input_ids']
    return inputs

tokenizer = MBartTokenizer.from_pretrained("./reduced_model", src_lang="ja_XX", tgt_lang="en_XX")
tokenizer.save_pretrained(result_dir)

In [ ]:
train_data = []
eval_data = []

for line in open("./split/train", "r", encoding='utf-8'):
    text = line.split('\t')
    train_data.append(
        {"translation": {
            "ja": text[1].rstrip('\n'),
            "en": text[0].rstrip('\n')
        }}
    )
print(f"train_data size: {len(train_data)}")

for line in open("./split/dev", "r", encoding='utf-8'):
    text = line.split('\t')
    eval_data.append(
        {"translation": {
            "ja": text[1].rstrip('\n'),
            "en": text[0].rstrip('\n')
        }}
    )
print(f"eval_data size: {len(eval_data)}")

In [ ]:
# Hyperparameters
batch_size = 1
learning_rate = 3e-5
epochs = 1

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("./reduced_model")

args = Seq2SeqTrainingArguments(output_dir=result_dir,
                                do_train=True,
                                do_eval=True,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                learning_rate=learning_rate,
                                num_train_epochs=epochs,
                                evaluation_strategy="epoch",
                                )

trainer = Seq2SeqTrainer(model=model,
                         args=args,
                         data_collator=data_collator,
                         train_dataset=train_data,
                         eval_dataset=eval_data,
                         )

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("./reduced_model")

args = Seq2SeqTrainingArguments(output_dir=result_dir,
                                do_train=True,
                                do_eval=True,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                learning_rate=learning_rate,
                                num_train_epochs=epochs,
                                evaluation_strategy="epoch",
                                )

trainer = Seq2SeqTrainer(model=model,
                         args=args,
                         data_collator=data_collator,
                         train_dataset=train_data,
                         eval_dataset=eval_data,
                         )

In [ ]:
trainer

In [ ]:
trainer.train()
trainer.save_model(result_dir)

In [ ]:
sentence = "おはよう"
inputs = tokenizer(sentence, return_tensors="pt")
translated_tokens = model.generate(**inputs, decoder_start_token_id=tokenizer.lang_code_to_id["en_XX"], early_stopping=True, max_length=48)
pred = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
print(f"日本語 - {sentence}: English - {pred}")